In [1]:
print('ok')

ok


In [2]:
%pwd

'c:\\Users\\User\\OneDrive\\Projects\\Medical-Chatbot-GenAI\\research'

In [3]:
import os
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\User\\OneDrive\\Projects\\Medical-Chatbot-GenAI'

In [5]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [6]:
#Extract data from the PDF file

def load_pdf_file(data):
    loader = DirectoryLoader(data,
                             glob="*.pdf",
                             loader_cls=PyPDFLoader)
    documents=loader.load()
    return documents


In [7]:
extracted_data = load_pdf_file(data="Data/")

In [8]:
#extracted_data

In [9]:
#Split the data into text chunks

def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

In [10]:
text_chunks = text_split(extracted_data)
print("Length of text chunks:", len(text_chunks))

Length of text chunks: 41255


In [11]:
from langchain.embeddings import HuggingFaceBgeEmbeddings

In [12]:
#Download the Embeddings from HUgginface

def download_huggingface_embeddings():
    embeddings=HuggingFaceBgeEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [13]:
import sentence_transformers
print(sentence_transformers.__version__)


c:\Users\User\anaconda3\envs\medibot\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


3.4.1


In [14]:
embeddings = download_huggingface_embeddings()

C:\Users\User\AppData\Local\Temp\ipykernel_25800\3265308951.py:4: LangChainDeprecationWarning: The class `HuggingFaceBgeEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings=HuggingFaceBgeEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')


In [15]:
query_result = embeddings.embed_query("What is the capital of France?")
print("Length", len(query_result))

Length 384


In [16]:
#query_result

In [34]:
from dotenv import load_dotenv
load_dotenv()

True

In [35]:
PINECONE_API_KEY =os.getenv("PINECONE_API_KEY")
OPENAI_API_KEY =os.getenv("OPENAI_API_KEY")

In [20]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec

import os

# Initialize Pinecone Client (New API)
pc = Pinecone(api_key=PINECONE_API_KEY)  # Replace with actual API key

index_name = "medicalbot"

# Create Index with `ServerlessSpec`
pc.create_index(
    name=index_name,
    dimension=384,  # Replace with your model dimensions
    metric="cosine",  # Replace with your model metric
    spec=ServerlessSpec(  # ✅ Required in Pinecone 6.0.1
        cloud="aws",
        region="us-east-1"
    )
)

In [36]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [24]:
#Embed each chunk and upsert the embeddings to the Pinecone Index
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    embedding=embeddings,
    index_name=index_name
)

In [25]:
#Load Existing Index
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_existing_index(
    embedding=embeddings,
    index_name=index_name
)

In [26]:
docsearch

In [27]:
retriever = docsearch.as_retriever(search_type ='similarity', search_kwargs={'k': 3})

In [28]:
retreived_docs = retriever.invoke("What is Acne?")

In [29]:
retreived_docs

[Document(id='9afe9f85-edd8-40ba-965b-3bcc18a708b4', metadata={'creationdate': '2006-10-16T20:19:33+02:00', 'creator': 'Adobe Acrobat 6.0', 'moddate': '2006-10-16T22:03:45+02:00', 'page': 54.0, 'page_label': '25', 'producer': 'PDFlib+PDI 6.0.3 (SunOS)', 'source': 'Data\\gale_encyclopedia_medicine.pdf', 'total_pages': 4505.0}, page_content='Acne vulgaris, the medical term for common acne,\nis the most common skin disease. It affects nearly 17\nmillion people in the United States. While acne can\narise at any age, it usually begins atpuberty and wor-\nsens during adolescence. Nearly 85% of people\ndevelop acne at some time between the ages of 12-25\nyears. Up to 20% of women develop mild acne. It is\nalso found in some newborns.\nThe sebaceous glands lie just beneath the skin’s\nsurface. They produce an oil called sebum, the skin’s'),
 Document(id='e1b9d692-d6a3-44df-834b-0c9ac3ff1b00', metadata={'creationdate': '2006-10-16T20:19:33+02:00', 'creator': 'Adobe Acrobat 6.0', 'moddate': '200

In [37]:
from langchain_openai import OpenAI
llm  = OpenAI(temperature=0.4, max_tokens=500)

In [38]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "Your are an assistant for questions answering on medical topics. "
    "Use the following pieces of retreived information to answer the question. "
    "If you do not know the answer, say that you do not know."
    "Use 3 sentences maximum to answer the question and keep the answer simple."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
   [
       ("system", system_prompt),
       ("human", "{input}"),
   ]
)

In [39]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [46]:
response = rag_chain.invoke({"input": "What is IBS?"})
print(response["answer"])



IBS is a condition that affects the large intestine and is characterized by inflammation. The cause of IBS is not fully understood, but it is believed to be a combination of dietary changes, drugs, and psychological factors. Women are more likely to have IBS than men and it can significantly impact daily life, causing individuals to miss work and school. Symptoms of IBS can vary in intensity.
